In [1]:
import pandas as pd
from pyspi.calculator import Calculator
import numpy as np
from copy import deepcopy
import glob
import os
import random
import neuromaps
from neuromaps.parcellate import Parcellater
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation
from pathlib import Path
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr
import statsmodels.stats.multitest

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

In [2]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(circlize)
    library(ComplexHeatmap)
    library(cowplot)
    library(dendextend)
    library(FactoMineR)
    library(GGally)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

# Load in data

In [5]:
data_path = "/Users/abry4213/data/HCP100/"

# Load in pyspi SPI info 
pyspi_info = pd.read_csv("../feature_extraction/pyspi_SPI_info.csv")

# Filter to sonnet SPIs
pyspi_info_sonnet = pyspi_info.query("Sonnet=='Yes'")

# Load in all the pyspi homotopic res
all_homotopic_res = (pd.concat([pd.read_csv(f) for f in glob.glob(data_path + "time_series_features/pyspi/*.csv")])
                     .merge(pyspi_info, on="SPI", how="left"))

# Filter to directed SPIs in sonnet
all_homotopic_res_mean = (all_homotopic_res.groupby(['base_region_to', 'SPI'])['value']
                              .mean()
                              .reset_index()
                              )

all_homotopic_res_mean

,base_region_to,SPI,value
0,bankssts,anm,0.308681
1,bankssts,bary-sq_dtw_max,10.019813
2,bankssts,bary-sq_dtw_mean,0.790094
3,bankssts,bary-sq_sgddtw_max,9.586980
4,bankssts,bary-sq_sgddtw_mean,0.713814
...,...,...,...
8699,transversetemporal,xme_gaussian_k10,1.368008
8700,transversetemporal,xme_kernel_W-0.5_k1,1.910185
8701,transversetemporal,xme_kernel_W-0.5_k10,-0.999781
8702,transversetemporal,xme_kozachenko_k1,1.370344


In [10]:
# Select di_gaussian out of all_homotopic_res_mean
di_gaussian = all_homotopic_res_mean.query("SPI=='di_gaussian'")
all_other_homotopic_res_mean = all_homotopic_res_mean.query("SPI!='di_gaussian'")
all_directed_homotopic_res_mean = all_other_homotopic_res_mean.merge(pyspi_info, on="SPI", how="left").query("Directionality=='Directed'")
all_undirected_homotopic_res_mean = all_other_homotopic_res_mean.merge(pyspi_info, on="SPI", how="left").query("Directionality=='Undirected'")

In [15]:
# Take all_undirected_homotopic_res_mean, group by SPI, and compute the Spearman correlation with di_gaussian (matched to base_region_to)
all_undirected_homotopic_res_mean_corrs = all_undirected_homotopic_res_mean.groupby("SPI").apply(lambda x: spearmanr(x['value'], di_gaussian['value'])[0]).reset_index()
all_undirected_homotopic_res_mean_corrs.sort_values(0, ascending=False)

/var/folders/rn/z0683sns3tz7b_k0cbf8ft8w0000gq/T/ipykernel_86811/4101110640.py:2: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  all_undirected_homotopic_res_mean_corrs = all_undirected_homotopic_res_mean.groupby("SPI").apply(lambda x: spearmanr(x['value'], di_gaussian['value'])[0]).reset_index()
/var/folders/rn/z0683sns3tz7b_k0cbf8ft8w0000gq/T/ipykernel_86811/4101110640.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_undirected_homotopic_res_mean_corrs = all_undirected_homotopic_res_mean.groupby("SPI").apply(lambda x: spearmanr(x['value'], di_gaussian['value'])[0]).reset_index()


,SPI,0
154,xcorr-sq_mean_sig-False,0.985943
120,prec-sq_EllipticEnvelope,0.983193
125,prec-sq_MinCovDet,0.983193
155,xcorr-sq_mean_sig-True,0.983193
82,mi_kraskov_NN-4,0.981360
...,...,...
106,pli_multitaper_mean_fs-1_fmin-0_fmax-0-25,NaN
107,pli_multitaper_mean_fs-1_fmin-0_fmax-0-5,NaN
150,wpli_multitaper_mean_fs-1_fmin-0-25_fmax-0-5,NaN
151,wpli_multitaper_mean_fs-1_fmin-0_fmax-0-25,NaN
